# Laboratorio 1 Phishing detection
## Security Data Science
### Universidad del valle de Guatemala

Abner Ivan Garcia - 21285
Oscar Esteba Donis - 21610


In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from yellowbrick.features import Rank2D
from yellowbrick.features import RadViz

In [2]:
df = pd.read_csv('dataset_pishing.csv')
print (df.shape)
print(df.head())
print(df.info())

# ejemplos de dominios del dataset (legitimos)
df[df.status == 'legitimate'].sample(10)

(11430, 2)
                                                 url      status
0              http://www.crestonwood.com/router.php  legitimate
1  http://shadetreetechnology.com/V4/validation/a...    phishing
2  https://support-appleld.com.secureupdate.duila...    phishing
3                                 http://rgipt.ac.in  legitimate
4  http://www.iracing.com/tracks/gateway-motorspo...  legitimate
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     11430 non-null  object
 1   status  11430 non-null  object
dtypes: object(2)
memory usage: 178.7+ KB
None


,url,status
322,https://www.drawdown.org/,legitimate
6327,http://www.mattiesaustin.com/,legitimate
8349,https://www.learnnext.com/user/signUpNew.htm,legitimate
8309,https://en.wikipedia.org/wiki/Computer_hardware,legitimate
1105,https://en.wikipedia.org/wiki/Internet_service...,legitimate
6589,https://www.uaoceu.es/,legitimate
3761,http://www.rokuhelp.com/tag/discovery-family/,legitimate
3932,https://www.walmart.com/c/kp/ethernet-networki...,legitimate
960,https://www.xhkzv.com/,legitimate
9872,http://www.brighttots.com/Autism/Self_help_skills,legitimate
